In [2]:
import boto3
import pandas as pd
from io import StringIO

In [3]:
s3 = boto3.resource('s3')

# Connecting to s3 bucket
bucket = s3.Bucket('xetra-1234')

In [4]:
# Inside the bucket there are a lot of folders, so we will connect to one of the with the name of foler as "2022-01-03"
bucket_obj1 = bucket.objects.filter(Prefix= '2022-01-03')

# We will create a 2nd bucket object as well so as to combine data available inside two buckets objects
    
    # Lets take the bucket object named "2022-01-04" so that when we combine data, we have 2 days worth of data combined in a single file.
bucket_obj2 = bucket.objects.filter(Prefix = "2022-01-04")

# Inside the bucket object again there are a lot of csv files, so we will create a variable with a list of all the files inside "2022-01-03"
    # objects1 is for all the files inside bucket_obj1
'''objects1 = [obj for obj in bucket_obj1]'''

    # objects2 is for all the files inside bucket_obj2
'''objects2 = [obj for obj in bucket_obj2]'''

# Or we can directly concatanate the list of two objects together

objects = [obj for obj in bucket_obj1] + [obj for obj in bucket_obj2]

In [5]:
# Checking whether joining the two lists is successful or not
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR10.csv'),
 s3.Object

* we can see that "objects" now contains the list of all the files from date "2022-01-03" and "2022-01-04"

In [6]:
# Lets get a list of column names of the data
csv_obj_init = bucket.Object(key = objects[0].key).get().get('Body').read().decode('utf-8')
data_init = StringIO(csv_obj_init)
df_init = pd.read_csv(data_init, delimiter = ',')

In [7]:
# chechking the culumn names in dataframe
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [8]:
# Creating an empty pandas dataframe and fixing the column names as per our dataset
df_merged = pd.DataFrame(columns=df_init.columns)

# Merging the data present in all the csv files into a single merged Dataset called df_merged using a for loop
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_merged = pd.concat([df_merged, df], ignore_index=True)

C:\Users\ankur\AppData\Local\Temp\ipykernel_15680\2990473283.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged = pd.concat([df_merged, df], ignore_index=True)
C:\Users\ankur\AppData\Local\Temp\ipykernel_15680\2990473283.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged = pd.concat([df_merged, df], ignore_index=True)
C:\Users\ankur\AppData\Local\Temp\ipykernel_15680\2990473283.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is depr

In [9]:
#lets check our merged dataset
df_merged

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414,2
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649,3
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011,36
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308,22
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239075,DE0007164600,SAP,SAP SE O.N.,Common stock,EUR,2505077,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000,3
239076,MT0000580101,M8G,MEDIA AND GAMES INV. EO 1,Common stock,EUR,2509636,2022-01-04,16:44,4.118,4.118,4.118,4.118,24,2
239077,DE000A0Z23Q5,ADN1,ADESSO SE INH O.N.,Common stock,EUR,2504440,2022-01-04,16:45,196.800,196.800,196.800,196.800,7,2
239078,DE000A3E5ES0,BIO0,BIOTEST AG ST O.N.Z.VERK.,Common stock,EUR,6973997,2022-01-04,20:30,43.000,43.000,43.000,43.000,0,1


In [10]:
# There are many columns which we do not need for further processing lets get rid of unwanted data
columns_to_included = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']

In [11]:
# Filtering our merged dataframe to only those columns which we need for further process
df_merged = df_merged.loc[:,columns_to_included]
df_merged

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414
1,DE000A0DJ6J9,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649
2,DE000A0D6554,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011
3,DE000A0D9PT0,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308
4,DE000A0HN5C6,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897
...,...,...,...,...,...,...,...,...
239075,DE0007164600,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000
239076,MT0000580101,2022-01-04,16:44,4.118,4.118,4.118,4.118,24
239077,DE000A0Z23Q5,2022-01-04,16:45,196.800,196.800,196.800,196.800,7
239078,DE000A3E5ES0,2022-01-04,20:30,43.000,43.000,43.000,43.000,0


* df_merged now containes only those columns which we need for further analysis

In [12]:
# Lets check our dataset for any Null Values
df_merged.isna().sum()

ISIN            0
Date            0
Time            0
StartPrice      0
MaxPrice        0
MinPrice        0
EndPrice        0
TradedVolume    0
dtype: int64

In [13]:
df_merged.shape

(239080, 8)